In [1]:
from __future__ import unicode_literals
import numpy as np
import pandas as pd
from fuzzywuzzy import fuzz
from subprocess import check_output
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize

from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, NuSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import metrics
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stopwords = stopwords.words('english')

import matplotlib.pyplot as plt
import seaborn as sns
import re
import os
import locale
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn import feature_extraction, linear_model, model_selection, preprocessing
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline

np.random.seed(1234)

os.environ["PYTHONIOENCODING"] = "utf-8"
# myLocale=locale.setlocale(category=locale.LC_ALL, locale="en_GB.UTF-8")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ali.faheem\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
path = os.path.join(".", r"D:\Faizad\Intelligent Computing Project\Results\RUHSOLD\dataset_old_non_repeat_combined.txt")
df = pd.read_csv(path, sep='\t') # trying to make the first row index_col=0
df.dropna(inplace=True)
combined_dataset = df[['Tweet','Score', 'Tag']]

# combined_dataset['label'] = combined_dataset['label'].replace([0],'normal').replace([1],'chutiya bc')

combined_dataset['Score'] = combined_dataset['Score'].replace([5,4],'overtly_hateful')
combined_dataset['Score'] = combined_dataset['Score'].replace([3,2,1],'hateful')
combined_dataset['Score'] = combined_dataset['Score'].replace([0],'no_hate')
combined_dataset['Tweet'] = combined_dataset['Tweet'].replace([0],'no_hate')

combined_dataset
data_frame = combined_dataset.copy()

In [3]:
data_frame.head(20)

,Tweet,Score,Tag
0,Chal hamare to shabdo ke mtlb ko badal k <Rel>...,overtly_hateful,Religious
1,<A/O> bhen chod Main sirf kotha khulna baki re...,overtly_hateful,Abusive/Offensive
2,<A/O> Bhen ki lodi <A/O> . Bigg boss nahi dekh...,overtly_hateful,Abusive/Offensive
3,<U> Bhen chod <U> twitter ko twitter kam <U> k...,hateful,Untargeted
4,<U> Bhen chod lun <U> ko tou mxakh smjh rakha ...,overtly_hateful,Untargeted
5,<A/O> Mai to maa chod du uski <A/O> jo meri ma...,overtly_hateful,Abusive/Offensive
6,"Main v Changa, Mere phone v changa, COD v chan...",hateful,Abusive/Offensive
7,Naahi us ki bhen ka yaar naahi magar itna pata...,overtly_hateful,Abusive/Offensive
8,Ye sanha ab bht over kr rhi ha bhen tu jitna m...,no_hate,Neutral
9,<A/O> Bhen k lund <A/O> <A/O> ye vahi hai jisn...,overtly_hateful,Religious


In [4]:
# data_frame

In [5]:
# path1 = os.path.join(".", "D:/Faizad/Intelligent Computing Project/Python_Notebooks/RUHSOLD_Dataset/New folder/Hate_Speech.txt")
# hate_speech_df = pd.read_csv(path1, header=None) # trying to make the first row index_col=0
# # hate_speech_df
# hate_speech_list = list(hate_speech_df[0])
# hate_speech = ' '.join(str(item) for item in hate_speech_list)

# path2 = os.path.join(".", "D:/Faizad/Intelligent Computing Project/Python_Notebooks/RUHSOLD_Dataset/New folder/Offensive.txt")
# Offensive_df = pd.read_csv(path2, header=None) # trying to make the first row index_col=0
# Offensive_list = list(Offensive_df[0])
# Offensive = ' '.join(str(item) for item in Offensive_list)

# path3 = os.path.join(".", "D:/Faizad/Intelligent Computing Project/Python_Notebooks/RUHSOLD_Dataset/New folder/Harassment .txt")
# Harassment_df = pd.read_csv(path3, header=None) # trying to make the first row index_col=0
# Harassment_list = list(Harassment_df[0])
# Harassment = ' '.join(str(item) for item in Harassment_list)

# path4 = os.path.join(".", "D:/Faizad/Intelligent Computing Project/Python_Notebooks/RUHSOLD_Dataset/New folder/Normal.txt")
# Normal_df = pd.read_csv(path4, header=None) # trying to make the first row index_col=0
# Normal_list = list(Normal_df[0])
# Normal = ' '.join(str(item) for item in Normal_list)

# path5 = os.path.join(".", "D:/Faizad/Intelligent Computing Project/Python_Notebooks/RUHSOLD_Dataset/New folder/Defamation.txt")
# Defamation_df = pd.read_csv(path5, header=None) # trying to make the first row index_col=0
# Defamation_list = list(Defamation_df[0])
# Defamation = ' '.join(str(item) for item in Defamation_list)

# path6 = os.path.join(".", "D:/Faizad/Intelligent Computing Project/Python_Notebooks/RUHSOLD_Dataset/New folder/Terrorism.txt")
# Terrorism_df = pd.read_csv(path6, header=None) # trying to make the first row index_col=0
# Terrorism_list = list(Terrorism_df[0])
# Terrorism = ' '.join(str(item) for item in Terrorism_list)

In [5]:
def extract_features(df):
    df['fuzz_ratio_hate_speech'] = df.apply(lambda x: fuzz.ratio(str(x['Tweet']), str(x['Tag'])), axis=1)
    df['fuzz_ratio_hate_speech'] = df['fuzz_ratio_hate_speech']/100
    
    df['fuzz_qratio_Offensive'] = df.apply(lambda x: fuzz.QRatio(str(x['Tweet']), str(x['Tag'])), axis=1)
    df['fuzz_qratio_Offensive'] = df['fuzz_qratio_Offensive'] / 100
    
    df['fuzz_uqratio_hate_speech'] = df.apply(lambda x: fuzz.UQRatio(str(x['Tweet']), str(x['Tag'])), axis=1)
    df['fuzz_uqratio_hate_speech'] = df['fuzz_uqratio_hate_speech'] / 100
    
    df['fuzz_uwratio_hate_speech'] = df.apply(lambda x: fuzz.UWRatio(str(x['Tweet']), str(x['Tag'])), axis=1) 
    df['fuzz_uwratio_hate_speech'] = df['fuzz_uwratio_hate_speech'] / 100
    
    df['fuzz_wratio_hate_speech'] = df.apply(lambda x: fuzz.WRatio(str(x['Tweet']), str(x['Tag'])), axis=1)
    df['fuzz_wratio_hate_speech'] = df['fuzz_wratio_hate_speech'] / 100
    
    df['fuzz_partial_ratio_hate_speech'] = df.apply(lambda x: fuzz.partial_ratio(str(x['Tweet']), str(x['Tag'])), axis=1)
    df['fuzz_partial_ratio_hate_speech'] = df['fuzz_partial_ratio_hate_speech'] / 100
    
    df['fuzz_partial_token_set_ratio_hate_speech'] = df.apply(lambda x: fuzz.partial_token_set_ratio(str(x['Tag']), str(x['Tag'])), axis=1)
    df['fuzz_partial_token_set_ratio_hate_speech'] = df['fuzz_partial_token_set_ratio_hate_speech'] / 100
    
    df['fuzz_partial_token_sort_ratio_hate_speech'] = df.apply(lambda x: fuzz.partial_token_sort_ratio(str(x['Tweet']), str(x['Tag'])), axis=1)
    df['fuzz_partial_token_sort_ratio_hate_speech'] = df['fuzz_partial_token_sort_ratio_hate_speech'] / 100
    
    df['fuzz_token_set_ratio_hate_speech'] = df.apply(lambda x: fuzz.token_set_ratio(str(x['Tweet']), str(x['Tag'])), axis=1)
    df['fuzz_token_set_ratio_hate_speech'] = df['fuzz_token_set_ratio_hate_speech'] / 100
    
    df['fuzz_token_sort_ratio_hate_speech'] = df.apply(lambda x: fuzz.token_sort_ratio(str(x['Tweet']), str(x['Tag'])), axis=1)
    df['fuzz_token_sort_ratio_hate_speech'] = df['fuzz_token_sort_ratio_hate_speech'] / 100
    
    return df
df = extract_features(data_frame)
# print (df)

In [7]:
# df.shape

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

# create an instance of the TfidfVectorizer class
vectorizer = TfidfVectorizer()

# fit the vectorizer to the documents and transform the documents into TF-IDF format
tfidf_matrix = vectorizer.fit_transform(df['Tweet'])

# print the resulting TF-IDF matrix
print(tfidf_matrix[0].toarray())

[[0. 0. 0. ... 0. 0. 0.]]


In [9]:
# from sklearn.feature_extraction.text import CountVectorizer

# # Create an instance of the CountVectorizer class
# vectorizer1 = CountVectorizer()

# # Fit the vectorizer to the documents and transform the documents into BoW format
# bow_matrix = vectorizer1.fit_transform(df['Tweet'])

# # Print the resulting BoW matrix
# print(bow_matrix[0].toarray())

In [10]:
# # Calculate the average word length for each tweet
# avg_word_length = df['Tweet'].apply(lambda x: np.mean([len(word) for word in x.split()]))

# # Convert average word length to a numpy array and reshape it as a column vector
# avg_word_length_vector = np.array(avg_word_length).reshape(-1, 1)

# # Print the average word length vector
# print(len(avg_word_length_vector))

In [11]:
# import numpy as np

# # Calculate the character count for each tweet
# char_count = df['Tweet'].apply(lambda x: len(x))

# # Convert character count to a numpy array and reshape it as a column vector
# char_count_vector = np.array(char_count).reshape(-1, 1)

# # Print the character count vector
# print((char_count_vector))

In [8]:
tfidf_df = pd.DataFrame(tfidf_matrix.toarray())

# bow_df = pd.DataFrame(bow_matrix.toarray())

# # awl_df = pd.DataFrame(avg_word_length_vector)

# # ccv_df = pd.DataFrame(char_count_vector)

In [13]:
# tfidf_df

In [9]:
feature_columns = df.columns.drop(['Tweet','Score','Tag'])
df1 = pd.DataFrame()
df1 = df[feature_columns]
# df1

# feature_columns = df.columns.drop(['sentence','label'])
# df1 = pd.DataFrame()
# df1 = df[feature_columns]
# # df1

In [10]:
concatenated_df = pd.concat([df1, tfidf_df], axis=1)
concatenated_df
# concatenated_df = pd.concat([tfidf_df, bow_df, awl_df, ccv_df,  df1], axis=1)

,fuzz_ratio_hate_speech,fuzz_qratio_Offensive,fuzz_uqratio_hate_speech,fuzz_uwratio_hate_speech,fuzz_wratio_hate_speech,fuzz_partial_ratio_hate_speech,fuzz_partial_token_set_ratio_hate_speech,fuzz_partial_token_sort_ratio_hate_speech,fuzz_token_set_ratio_hate_speech,fuzz_token_sort_ratio_hate_speech,...,22308,22309,22310,22311,22312,22313,22314,22315,22316,22317
0,0.05,0.05,0.05,0.26,0.26,0.44,1.0,0.44,0.06,0.05,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.14,0.14,0.14,0.23,0.23,0.29,1.0,0.41,0.19,0.17,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.10,0.11,0.11,0.21,0.21,0.35,1.0,0.35,0.17,0.14,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.16,0.17,0.17,0.27,0.27,0.30,1.0,0.30,0.19,0.18,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.15,0.16,0.16,0.34,0.34,0.20,1.0,0.40,0.17,0.15,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10003,0.15,0.16,0.16,0.36,0.36,0.40,1.0,0.40,0.14,0.14,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10004,0.06,0.07,0.07,0.26,0.26,0.43,1.0,0.43,0.07,0.07,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10005,0.10,0.13,0.13,0.26,0.26,0.43,1.0,0.43,0.11,0.10,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10006,0.10,0.11,0.11,0.26,0.26,0.29,1.0,0.29,0.09,0.08,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
# feature_columns = df.columns.drop(['Score','Tag'])

X_normalized = normalize(concatenated_df, norm='l2', axis=1, copy=True, return_norm=False)
# print(X_normalized)

x_train, x_test, y_train, y_test = train_test_split(X_normalized, df['Tag'], random_state=1, test_size=0.2)

result_cols = ["Classifier", "Accuracy"]
result_frame = pd.DataFrame(columns=result_cols)

In [13]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import classification_report

classifiers = [
#     DecisionTreeClassifier(),
    RandomForestClassifier(),
#     LogisticRegression(),
    KNeighborsClassifier(),
#     AdaBoostClassifier(),
#     GaussianNB(),
#     SVC(gamma='auto'),
    ]    

In [14]:
for clf in classifiers:
    name = clf.__class__.__name__
    clf.fit(x_train, y_train)
    predicted = clf.predict(x_test)
    print(classification_report(y_test,predicted))
#     classification_rep = classification_report(y_test,predicted)
#     output_folder = "D:/Faizad/Intelligent Computing Project/Results/RUHSOLD/TF-IDF"
#     output_filename = name+('.txt')
#     output_path = os.path.join(output_folder, output_filename)
#     with open(output_path, "w", encoding='utf-8') as f:
#         f.write(classification_rep)
    acc = metrics.accuracy_score(y_test,predicted)
    print (name +' accuracy = '+str(acc*100)+'%')
    acc_field = pd.DataFrame([[name, acc*100]], columns=result_cols)
    result_frame = result_frame.append(acc_field)

                   precision    recall  f1-score   support

Abusive/Offensive       0.93      0.81      0.87       497
          Neutral       0.86      0.99      0.92      1081
        Religious       0.98      0.92      0.95       165
           Sexist       0.94      0.75      0.84       159
       Untargeted       0.93      0.43      0.59       100

         accuracy                           0.89      2002
        macro avg       0.93      0.78      0.83      2002
     weighted avg       0.90      0.89      0.89      2002

RandomForestClassifier accuracy = 89.21078921078922%


C:\Users\ali.faheem\AppData\Local\Temp\ipykernel_2180\775732946.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_frame = result_frame.append(acc_field)


                   precision    recall  f1-score   support

Abusive/Offensive       0.71      0.59      0.64       497
          Neutral       0.77      0.90      0.83      1081
        Religious       0.81      0.87      0.84       165
           Sexist       0.55      0.49      0.52       159
       Untargeted       0.48      0.10      0.17       100

         accuracy                           0.74      2002
        macro avg       0.66      0.59      0.60      2002
     weighted avg       0.73      0.74      0.73      2002

KNeighborsClassifier accuracy = 74.47552447552448%


C:\Users\ali.faheem\AppData\Local\Temp\ipykernel_2180\775732946.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_frame = result_frame.append(acc_field)


In [ ]:
# def extract_features(df):
# #     df = []
# #     df['fuzz_qratio_hate_speech'] = df.apply(lambda x: fuzz.QRatio(str(x['Tweet']), str(hate_speech+Offensive+Harassment+Normal+Defamation+Terrorism)), axis=1)
#     df['fuzz_qratio_hate_speech'] = df.apply(lambda x: fuzz.ratio(str(x['Tweet']), str(x['Tag'])), axis=1)
#     df['fuzz_qratio_hate_speech'] = df['fuzz_qratio_hate_speech']/100
    
#     df['fuzz_qratio_Offensive'] = df.apply(lambda x: fuzz.ratio(str(x['Tweet']), str(Offensive)), axis=1)
#     df['fuzz_qratio_Offensive'] = df['fuzz_qratio_Offensive']/100
    
#     df['fuzz_qratio_Harassment'] = df.apply(lambda x: fuzz.ratio(str(x['Tweet']), str(Harassment)), axis=1)
#     df['fuzz_qratio_Harassment'] = df['fuzz_qratio_Harassment'] /100
    
#     df['fuzz_qratio_Normal'] = df.apply(lambda x: fuzz.ratio(str(x['Tweet']), str(Normal)), axis=1)
#     df['fuzz_qratio_Normal'] = df['fuzz_qratio_Normal'] /100
    
#     df['fuzz_qratio_Defamation'] = df.apply(lambda x: fuzz.ratio(str(x['Tweet']), str(Defamation)), axis=1)
#     df['fuzz_qratio_Defamation'] = df['fuzz_qratio_Defamation'] /100
    
#     df['fuzz_qratio_Terrorism'] = df.apply(lambda x: fuzz.ratio(str(x['Tweet']), str(Terrorism)), axis=1)
#     df['fuzz_qratio_Terrorism'] = df['fuzz_qratio_Terrorism'] /100
    
# #     df['fuzz_uqratio_hate_speech'] = df.apply(lambda x: fuzz.UQRatio(str(x['Tweet']), str(hate_speech)), axis=1)
#     df['fuzz_uqratio_Offensive'] = df.apply(lambda x: fuzz.UQRatio(str(x['Tweet']), str(Offensive)), axis=1)
# #     df['fuzz_uqratio_Harassment'] = df.apply(lambda x: fuzz.UQRatio(str(x['Tweet']), str(Harassment)), axis=1)
# #     df['fuzz_uqratio_Normal'] = df.apply(lambda x: fuzz.UQRatio(str(x['Tweet']), str(Normal)), axis=1)
# #     df['fuzz_uqratio_Defamation'] = df.apply(lambda x: fuzz.UQRatio(str(x['Tweet']), str(Defamation)), axis=1)
# #     df['fuzz_uqratio_Terrorism'] = df.apply(lambda x: fuzz.UQRatio(str(x['Tweet']), str(Terrorism)), axis=1)
    
# #     df['fuzz_uwratio_hate_speech'] = df.apply(lambda x: fuzz.UWRatio(str(x['Tweet']), str(hate_speech)), axis=1)
# #     df['fuzz_uwratio_Offensive'] = df.apply(lambda x: fuzz.UWRatio(str(x['Tweet']), str(Offensive)), axis=1)
#     df['fuzz_uwratio_Harassment'] = df.apply(lambda x: fuzz.UWRatio(str(x['Tweet']), str(Harassment)), axis=1)
# #     df['fuzz_uwratio_Normal'] = df.apply(lambda x: fuzz.UWRatio(str(x['Tweet']), str(Normal)), axis=1)
# #     df['fuzz_uwratio_Defamation'] = df.apply(lambda x: fuzz.UWRatio(str(x['Tweet']), str(Defamation)), axis=1)
# #     df['fuzz_uwratio_Terrorism'] = df.apply(lambda x: fuzz.UWRatio(str(x['Tweet']), str(Terrorism)), axis=1)
    
# #     df['fuzz_wratio_hate_speech'] = df.apply(lambda x: fuzz.WRatio(str(x['Tweet']), str(hate_speech)), axis=1)
# #     df['fuzz_wratio_Offensive'] = df.apply(lambda x: fuzz.WRatio(str(x['Tweet']), str(Offensive)), axis=1)
# #     df['fuzz_wratio_Harassment'] = df.apply(lambda x: fuzz.WRatio(str(x['Tweet']), str(Harassment)), axis=1)
#     df['fuzz_wratio_Normal'] = df.apply(lambda x: fuzz.WRatio(str(x['Tweet']), str(Normal)), axis=1)
# #     df['fuzz_wratio_Defamation'] = df.apply(lambda x: fuzz.WRatio(str(x['Tweet']), str(Defamation)), axis=1)
# #     df['fuzz_wratio_Terrorism'] = df.apply(lambda x: fuzz.WRatio(str(x['Tweet']), str(Terrorism)), axis=1)
    
#     df['fuzz_partial_ratio_hate_speech'] = df.apply(lambda x: fuzz.partial_ratio(str(x['Tweet']), str(x['Tag'])), axis=1)
#     df['fuzz_partial_ratio_hate_speech'] = df['fuzz_partial_ratio_hate_speech'] / 100
# # #     df['fuzz_partial_ratio_Offensive'] = df.apply(lambda x: fuzz.partial_ratio(str(x['Tweet']), str(Offensive)), axis=1)
# # #     df['fuzz_partial_ratio_Harassment'] = df.apply(lambda x: fuzz.partial_ratio(str(x['Tweet']), str(Harassment)), axis=1)
# # #     df['fuzz_partial_ratio_Normal'] = df.apply(lambda x: fuzz.partial_ratio(str(x['Tweet']), str(Normal)), axis=1)
# #     df['fuzz_partial_ratio_Defamation'] = df.apply(lambda x: fuzz.partial_ratio(str(x['Tweet']), str(Defamation)), axis=1)
# # #     df['fuzz_partial_ratio_Terrorism'] = df.apply(lambda x: fuzz.partial_ratio(str(x['Tweet']), str(Terrorism)), axis=1)
    
#     df['fuzz_partial_token_set_ratio_hate_speech'] = df.apply(lambda x: fuzz.partial_token_set_ratio(str(x['Tweet']), str(x['Tag'])), axis=1)
#     df['fuzz_partial_token_set_ratio_hate_speech'] = df['fuzz_partial_token_set_ratio_hate_speech'] / 100
# #     df['fuzz_partial_token_set_ratio_Offensive'] = df.apply(lambda x: fuzz.partial_token_set_ratio(str(x['Tweet']), str(Offensive)), axis=1)
# #     df['fuzz_partial_token_set_ratio_Harassment'] = df.apply(lambda x: fuzz.partial_token_set_ratio(str(x['Tweet']), str(Harassment)), axis=1)
# #     df['fuzz_partial_token_set_ratio_Normal'] = df.apply(lambda x: fuzz.partial_token_set_ratio(str(x['Tweet']), str(Normal)), axis=1)
# #     df['fuzz_partial_token_set_ratio_Defamation'] = df.apply(lambda x: fuzz.partial_token_set_ratio(str(x['Tweet']), str(Defamation)), axis=1)
# #     df['fuzz_partial_token_set_ratio_Terrorism'] = df.apply(lambda x: fuzz.partial_token_set_ratio(str(x['Tweet']), str(Terrorism)), axis=1)
    
#     df['fuzz_partial_token_sort_ratio_hate_speech'] = df.apply(lambda x: fuzz.partial_token_sort_ratio(str(x['Tweet']), str(x['Tag'])), axis=1)
#     df['fuzz_partial_token_sort_ratio_hate_speech'] = df['fuzz_partial_token_sort_ratio_hate_speech'] / 100
# #     df['fuzz_partial_token_sort_ratio_Offensive'] = df.apply(lambda x: fuzz.partial_token_sort_ratio(str(x['Tweet']), str(Offensive)), axis=1)
# #     df['fuzz_partial_token_sort_ratio_Harassment'] = df.apply(lambda x: fuzz.partial_token_sort_ratio(str(x['Tweet']), str(Harassment)), axis=1)
# #     df['fuzz_partial_token_sort_ratio_Normal'] = df.apply(lambda x: fuzz.partial_token_sort_ratio(str(x['Tweet']), str(Normal)), axis=1)
# #     df['fuzz_partial_token_sort_ratio_Defamation'] = df.apply(lambda x: fuzz.partial_token_sort_ratio(str(x['Tweet']), str(Defamation)), axis=1)
# #     df['fuzz_partial_token_sort_ratio_Terrorism'] = df.apply(lambda x: fuzz.partial_token_sort_ratio(str(x['Tweet']), str(Terrorism)), axis=1)
    
#     df['fuzz_token_set_ratio_hate_speech'] = df.apply(lambda x: fuzz.token_set_ratio(str(x['Tweet']), str(x['Tag'])), axis=1)
#     df['fuzz_token_set_ratio_hate_speech'] = df['fuzz_token_set_ratio_hate_speech'] / 100
# #     df['fuzz_token_set_ratio_Offensive'] = df.apply(lambda x: fuzz.token_set_ratio(str(x['Tweet']), str(Offensive)), axis=1)
# # #     df['fuzz_token_set_ratio_Harassment'] = df.apply(lambda x: fuzz.token_set_ratio(str(x['Tweet']), str(Harassment)), axis=1)
# # #     df['fuzz_token_set_ratio_Normal'] = df.apply(lambda x: fuzz.token_set_ratio(str(x['Tweet']), str(Normal)), axis=1)
# # #     df['fuzz_token_set_ratio_Defamation'] = df.apply(lambda x: fuzz.token_set_ratio(str(x['Tweet']), str(Defamation)), axis=1)
# # #     df['fuzz_token_set_ratio_Terrorism'] = df.apply(lambda x: fuzz.token_set_ratio(str(x['Tweet']), str(Terrorism)), axis=1)
    
#     df['fuzz_token_sort_ratio_hate_speech'] = df.apply(lambda x: fuzz.token_sort_ratio(str(x['Tweet']), str(x['Tag'])), axis=1)
#     df['fuzz_token_sort_ratio_hate_speech'] = df['fuzz_token_sort_ratio_hate_speech'] / 100
# # #     df['fuzz_token_sort_ratio_Offensive'] = df.apply(lambda x: fuzz.token_sort_ratio(str(x['Tweet']), str(Offensive)), axis=1)
# #     df['fuzz_token_sort_ratio_Harassment'] = df.apply(lambda x: fuzz.token_sort_ratio(str(x['Tweet']), str(Harassment)), axis=1)
# # #     df['fuzz_token_sort_ratio_Normal'] = df.apply(lambda x: fuzz.token_sort_ratio(str(x['Tweet']), str(Normal)), axis=1)
# # #     df['fuzz_token_sort_ratio_Defamation'] = df.apply(lambda x: fuzz.token_sort_ratio(str(x['Tweet']), str(Defamation)), axis=1)
# # #     df['fuzz_token_sort_ratio_Terrorism'] = df.apply(lambda x: fuzz.token_sort_ratio(str(x['Tweet']), str(Terrorism)), axis=1)

#     return df
# df = extract_features(data_frame)
# # print (df)

In [ ]:
# # feature_columns = df.columns.drop(['Score','Tag'])

# X_normalized = normalize(df[feature_columns].toarray(), norm='l2',axis=1, copy=True, return_norm=False)
# # print (X_normalized)
# x_train, x_test, y_train, y_test = train_test_split(X_normalized, df['Tag'], random_state = 1,test_size=0.2)

# result_cols = ["Classifier", "Accuracy"]
# result_frame = pd.DataFrame(columns=result_cols)



# feature_columns = df.columns.drop(['Score','Tag'])

X_normalized = normalize(bow_matrix, norm='l2', axis=1, copy=True, return_norm=False)
# print(X_normalized)

x_train, x_test, y_train, y_test = train_test_split(X_normalized, df['Tag'], random_state=1, test_size=0.2)

result_cols = ["Classifier", "Accuracy"]
result_frame = pd.DataFrame(columns=result_cols)


In [ ]:
# # feature_columns = df.columns.drop(['Score','Tag'])

# X_normalized = normalize(df[feature_columns].toarray(), norm='l2',axis=1, copy=True, return_norm=False)
# # print (X_normalized)
# x_train, x_test, y_train, y_test = train_test_split(X_normalized, df['Tag'], random_state = 1,test_size=0.2)

# result_cols = ["Classifier", "Accuracy"]
# result_frame = pd.DataFrame(columns=result_cols)



# feature_columns = df.columns.drop(['Score','Tag'])

X_normalized = normalize(df[feature_columns], norm='l2', axis=1, copy=True, return_norm=False)
# print(X_normalized)

x_train, x_test, y_train, y_test = train_test_split(X_normalized, df['Tag'], random_state=1, test_size=0.2)

result_cols = ["Classifier", "Accuracy"]
result_frame = pd.DataFrame(columns=result_cols)


In [ ]:
# sns.set_color_codes("muted")
# sns.barplot(x='Accuracy', y='Classifier', data=result_frame, color="r")

# plt.xlabel('Accuracy %')
# plt.title('Classifier Accuracy')
# plt.show()

In [ ]:
############################################

In [ ]:
############################################